# Applications of MLP

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Churn Analysis
---
- Churn refers to the number of customers who stop doing business with a company within a given time period. <br><br>
- A high churn rate can be a sign of customer dissatisfaction or a problem with the company's products or services. It can also be caused by factors outside of the company's control, such as a competitor offering a better deal.<br><br>
- Churning customers can have a number of economic disadvantages for a business, including:<br>
    - Loss of revenue. When customers churn, they stop paying for the company's products or services. This can lead to a significant loss of revenue, especially for businesses that rely on recurring revenue.<br><br>
    - Increased marketing costs. To replace lost customers, businesses often need to spend more on marketing and sales. This can be a significant expense, especially for businesses with a high churn rate.<br><br>
    - Damage to brand reputation. When customers churn, they may spread negative word-of-mouth about the company. This can damage the company's brand reputation and make it more difficult to attract new customers.<br><br>
    - Increased customer acquisition costs. It is more expensive to acquire new customers than to retain existing customers. This is because new customers are less likely to be familiar with the company's products or services and may require more hand-holding.<br><br>
    - Decreased customer lifetime value. The customer lifetime value (CLV) is the total amount of money that a customer is expected to spend with a company over their lifetime. When customers churn, the company loses out on the potential future revenue that they could have generated from that customer.<br><br>

- The economic disadvantages of churning customers can be significant, so it is important for businesses to take steps to reduce churn. Some common ways to reduce churn include:<br><br>

    - Providing excellent customer service. This means being responsive to customer inquiries, resolving problems quickly and efficiently, and going the extra mile to make sure that customers are satisfied.<br><br>
    - Offering competitive prices. This means keeping prices in line with the competition and offering discounts and promotions to encourage customers to stay with the company.<br><br>
    - Constantly innovating. This means keeping up with the latest trends and offering new and improved products and services to keep customers engaged.<br><br>
    - Personalizing the customer experience. This means getting to know each customer's individual needs and preferences and tailoring the company's offerings accordingly.
<br><br>
- By taking steps to reduce churn, businesses can improve their bottom line and ensure long-term success.

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('bank_data.csv')

In [ ]:
dataset

In [ ]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
pd.DataFrame(X, columns = ["CreditScore","Geography","Gender","Age","Tenure","Balance","NumOfProducts","HasCrCard","IsActiveMember","EstimatedSalary"])

In [ ]:
pd.DataFrame(y, columns = ["Exited"])

In [ ]:
X

In [ ]:
y

### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer


In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
accuracy_score(y_test, y_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

**Homework**
- Use the dataset: https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset.
- Experiment with the number of hidden layers, and the number of units in the hidden layer.
- Try to experiment on the activation functions in the hidden layer https://www.tensorflow.org/api_docs/python/tf/keras/activations
- Try to experiment with the optimizer: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
- Check the accuracy, specificity, and sensitivity of your model.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import SGD, Adam, RMSprop

# Loading and preprocessing data based on their data type
dataset = pd.read_csv('diabetes_prediction_dataset.csv')

# Data Encoding
binary_columns = ['hypertension', 'heart_disease', 'diabetes']
dataset[binary_columns] = dataset[binary_columns].apply(LabelEncoder().fit_transform)

nominal_columns = ['gender', 'smoking_history']
dataset = pd.get_dummies(dataset, columns=nominal_columns, drop_first=True)

numerical_columns = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
dataset[numerical_columns] = StandardScaler().fit_transform(dataset[numerical_columns])

X = dataset.drop(columns=['diabetes'])
y = dataset['diabetes']

# Splitting data into testing and training data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function that will create and train a model based on the parameters passed
def create_and_train_model(units_per_layer, activation_function, optimizer):
    tf.config.run_functions_eagerly(False) 
    model = Sequential()
    model.add(Dense(units=units_per_layer[0], activation=activation_function, input_dim=X_train.shape[1]))

    for units in units_per_layer[1:]:
        model.add(Dense(units=units, activation=activation_function))
    
    model.add(Dense(units=1, activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Loss: {loss}, Accuracy: {accuracy}")

# Chosen configurations from the tensorflow documentation
units_per_layer_list = [[32], [64, 32], [128, 64, 32]] 
activation_functions = ['sigmoid', 'relu', 'tanh'] 
optimizers = [SGD(learning_rate=0.01), Adam(learning_rate=0.001), RMSprop(learning_rate=0.001)]  

# Loop that will call the function with different configurations
for units_per_layer in units_per_layer_list:
    for activation_function in activation_functions:
        for optimizer in optimizers:
            tf.keras.backend.clear_session()
            print(f"Units per Layer: {units_per_layer}, Activation: {activation_function}, Optimizer: {optimizer.get_config()['name']}")
            create_and_train_model(units_per_layer, activation_function, optimizer)

# Based on the results, the configuration that yieled the highest accuracy while having the least loss is:
# Units per Layer: [128, 64, 32], Activation: relu, Optimizer: Adam
# With the loss value of: 0.08195851743221283 and Accuracy of: 0.9721999764442444
